# EXTENSION DEL API DE PANDAS PARA MANEJO DE DATOS FALTANTES(MISSING)

In [3]:
!pip install upsetplot

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for upsetplot: filename=UpSetPlot-0.9.0-py3-none-any.whl size=24839 sha256=262c3bcb2bfc23b1b11afaa13eb1f395d8aa2b523de66a65d13b3507462cac33
  Stored in directory: /root/.cache/pip/wheels/7b/ce/88/1d0bce5b2680165c29c3e68416325e79674d5a9f422d037996
Successfully built upsetplot


In [4]:
import itertools
import upsetplot
import pandas as pd

In [5]:
try:
  del pd.DataFrame.missing
except AttributeError:
  pass

In [6]:
@pd.api.extensions.register_dataframe_accessor("missing")
class MissingMethods:
    def __init__(self,pandas_obj):
      self._obj = pandas_obj

    def number_missing(self) -> int:
      return self._obj.isna().sum().sum()

    def number_complete(self) -> int:
      return self._obj.size - self._obj.missing.number_missing()

    def missing_variable_summary(self) -> pd.DataFrame:
        return self._obj.isna().pipe(
            lambda df_1: (
                df_1.sum()
                .reset_index(name="n_missing")
                .rename(columns={"index":"variable"})
                .assign(
                    n_cases=len(df_1),
                    pct_missing=lambda df_2: df_2.n_missing / df_2.n_cases * 100,
                )
            )
        )